In [2]:
from dabry.problem import NavigationProblem
from dabry.wind import DiscreteWind
from dabry.model import ZermeloGeneralModel
from dabry.misc import Utils
from dabry.ddf_manager import DDFmanager
from dabry.solver_ef import SolverEF
import numpy as np

In [14]:
i_member = 0
for i_member in range(51):
    wind = DiscreteWind()
    wind.load(f'../output/wind_scenarii2_{i_member}/wind.h5')
    model = ZermeloGeneralModel(23, coords='gcs')
    model.update_wind(wind)
    
    x_init = Utils.DEG_TO_RAD * np.array((55, -23))
    x_target = Utils.DEG_TO_RAD * np.array((116, -32))

    bl = Utils.DEG_TO_RAD * np.array((42, -43))
    tr = Utils.DEG_TO_RAD * np.array((125, 0))

    pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

    ddf = DDFmanager(cache_wind=False)
    ddf.setup()
    ddf.set_case(f'wind_scenarii2_{i_member}')
    ddf.clean_output_dir()
    ddf.dump_wind(pb.model.wind)
    
    solver = SolverEF(pb, max_time=20*3600, max_steps=500)
    res = solver.solve()
    
    ddf.dump_trajs([res.traj])
    #ddf.dump_trajs(solver.get_trajs())
    ddf.log(pb)

Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 105h0m
Steps :    142/500, Extremals :    303, Active :   43, Dist :  89 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 112h38m
Steps :    260/500, Extremals :    373, Active :  152, Dist :  11 Stopped empty active list
Target reached in 105h33m | +44% no wind | -6% orthodromic | cpu time 32.11s
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 106h18m
Steps :    102/500, Extremals :    229, Active :   48, Dist : 100 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 113h50m
Steps :    254/500, Extremals :    378, Active :  222, Dist :  12 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 112h3m
Steps :     20/500, Extremals :     79, Active : 

Steps :     30/500, Extremals :    106, Active :  106, Dist :  83 

Steps :    266/500, Extremals :    354, Active :  180, Dist :  11 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 105h39m
Steps :    162/500, Extremals :    266, Active :   84, Dist :  18 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 104h33m
Steps :     92/500, Extremals :    225, Active :   37, Dist : 100 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 108h38m
Steps :    112/500, Extremals :    255, Active :   33, Dist : 100 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 183h53m
Steps :     30/500, Extremals :    108, Active :  108, Dist :  83 

Steps :    289/500, Extremals :    422, Active :   69, Dist :  20 Stopped empty active list
No solution found in time < 285h18m
Problem | TIMEOPT | 23.00 m/s | 6.03e+06 m | scale 72h48m | orthodromic 111h31m
Steps :     60/500, Extremals :    172, Active :  172, Dist :  59 

AttributeError: 'NoneType' object has no attribute 'distance'

In [7]:
import shutil
for i_member in range(51):
    try:
        shutil.copy(f'../output/wind_scenarii2_{i_member}/trajectories.h5',
                   f'../output/wind_scenarii2_all/trajectories_{i_member}.h5')
    except FileNotFoundError:
        pass
shutil.copy('../output/wind_scenarii2_0/params.json', '../output/wind_scenarii2_all/params.json')
shutil.copy('../output/wind_scenarii2_0/wind.h5', '../output/wind_scenarii2_all/wind.h5')

'../output/wind_scenarii2_all/wind.h5'

In [15]:
wind = DiscreteWind()
wind.load(f'../output/wind_scenarii_all/wind.h5')
model = ZermeloGeneralModel(23, coords='gcs')
model.update_wind(wind)

x_init = Utils.DEG_TO_RAD * np.array((-55, 40))
x_target = Utils.DEG_TO_RAD * np.array((-10, 40))

bl = Utils.DEG_TO_RAD * np.array((-60, 33))
tr = Utils.DEG_TO_RAD * np.array((-5, 55))

pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

ddf = DDFmanager(cache_wind=True)
ddf.setup()
ddf.set_case(f'wind_scenarii_all')

from dabry.feedback import HTargetFB
from dabry.stoppingcond import DistanceSC
pb.load_feedback(HTargetFB(x_target, 'gcs'))
sc = DistanceSC(lambda x: (x - x_target) @ (x - x_target), (x_init - x_target) @ (x_init - x_target) / 500)
pb.integrate_trajectory(x_init, sc, t_init=pb.model.wind.t_start, int_step=3600/100)
ddf.dump_trajs(pb.trajs, filename='trajectories_heuristic.h5')
ddf.log(pb)